# Homework -- classify queries to their item type

We have pre-enriched products assigned with an item type. Below is a skeleton query classifier for the item type. What NDCG can you achieve my changing the query side?

Your task is to:

1. Use the provided query ground truth for item types
2. Change your classifier to preduct that ground truth
3. Use the search strategy to improve NDCG

Before you begin, just run through the notebook once. It should run. THEN

Change the prompt and prompting strategy to try to improve precision against the ground truth.

## Boilerplate

Install deps, mount GDrive, prompt for your OpenAI Key (placed in your GDrive)

In [ ]:
!pip install git+https://github.com/softwaredoug/cheat-at-search.git
from cheat_at_search.data_dir import mount
mount(use_gdrive=True)
from cheat_at_search.search import run_strategy, graded_bm25, ndcgs, ndcg_delta, vs_ideal
from cheat_at_search.wands_data import products, enriched_products, enriched_queries
from cheat_at_search.cache import StoredLruCache


from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

  Cloning https://github.com/softwaredoug/cheat-at-search.git to /tmp/pip-req-build-_24nl_tj
  Running command git clone --filter=blob:none --quiet https://github.com/softwaredoug/cheat-at-search.git /tmp/pip-req-build-_24nl_tj
  Resolved https://github.com/softwaredoug/cheat-at-search.git to commit 7fbf2bf2845343912918f337a29127f9edc50bd2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.3/745.3 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 85.0 MB/s eta 0:00:00
  Created wheel for cheat_at_search: filename=cheat_at_search-0.1.0-py3-none-any.whl size=28125 sha256=d79ddefa4e5881651692ac6a334752165c4fd3a5b606f8abee953f566b5d58ce
  Stored in directory: /tmp/pip-ephem-wheel-cache-fekul8dn/wheels/ea/96/d3/e69453e5f86e6f891864e5b5baf2b11ffda2d0981d9

INFO:data_dir:Mounting data directory at /content/drive/MyDrive/cheat-at-search-data/


2025-09-01 13:26:41,807 - data_dir - INFO - Directory /content/drive/MyDrive/cheat-at-search-data/wands_enriched already exists. Checking for updates...


INFO:data_dir:Directory /content/drive/MyDrive/cheat-at-search-data/wands_enriched already exists. Checking for updates...


2025-09-01 13:27:16,197 - data_dir - INFO - Updated https://github.com/softwaredoug/WANDS.git dataset at /content/drive/MyDrive/cheat-at-search-data/wands_enriched


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Review item types classified

In a [previous example notebook](https://colab.research.google.com/drive/1S6GdDMN-I4wFY4obWLs0STNlT_7mKrXM) we classified items to item types. Below you can see they've been precomputed for you to use as an attribute on every product.

In [ ]:
enriched_products

,product_id,product_name,product_class,category hierarchy,product_description,product_features,rating_count,average_rating,review_count,features,...,sub_category,cat_subcat,item_type_sim,item_type_unconstrained,item_type,item_type_same,branded_terms,room,materials,classification
0,0,solid wood platform bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,"good , deep sleep can be quite difficult to ha...",overallwidth-sidetoside:64.7|dsprimaryproducts...,15.0,4.5,15.0,"['overallwidth-sidetoside:64.7', 'dsprimarypro...",...,Bedroom Furniture,Furniture / Bedroom Furniture,0.625533,platform bed,bed,bed,solid wood platform bed,bedroom,acacia wood,Furniture / Bedroom Furniture / Beds & Headboa...
1,1,all-clad 7 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,"create delicious slow-cooked meals , from tend...",capacityquarts:7|producttype : slow cooker|pro...,100.0,2.0,98.0,"['capacityquarts:7', 'producttype : slow cooke...",...,Small Kitchen Appliances,Kitchen & Tabletop / Small Kitchen Appliances,1.000000,slow cooker,slow cooker,slow cooker,7 qt sep all-clad sep slow cooker,kitchen,nonstick cast-aluminum sep stainless steel,Kitchen & Tabletop / Small Kitchen Appliances ...
2,2,all-clad electrics 6.5 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,prepare home-cooked meals on any schedule with...,features : keep warm setting|capacityquarts:6....,208.0,3.0,181.0,"['features : keep warm setting', 'capacityquar...",...,Small Kitchen Appliances,Kitchen & Tabletop / Small Kitchen Appliances,1.000000,slow cooker,slow cooker,slow cooker,all-clad electrics sep all-clad,kitchen,,Kitchen & Tabletop / Small Kitchen Appliances ...
3,3,all-clad all professional tools pizza cutter,"Slicers, Peelers And Graters",Browse By Brand / All-Clad,this original stainless tool was designed to c...,overallwidth-sidetoside:3.5|warrantylength : l...,69.0,4.5,42.0,"['overallwidth-sidetoside:3.5', 'warrantylengt...",...,All-Clad,Browse By Brand / All-Clad,0.047557,pizza cutter,no item type matches,no item type matches,all-clad sep all professional tools,kitchen,stainless steel,Browse By Brand / All-Clad
4,4,baldwin prestige alcott passage knob with roun...,Door Knobs,Home Improvement / Doors & Door Hardware / Doo...,the hardware has a rich heritage of delivering...,compatibledoorthickness:1.375 '' |countryofori...,70.0,5.0,42.0,"[""compatibledoorthickness:1.375 '' "", 'country...",...,Doors & Door Hardware,Home Improvement / Doors & Door Hardware,1.000000,door knob,door knob,door knob,prestige alcott sep baldwin,unknown,,Home Improvement / Doors & Door Hardware / Doo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42989,42989,malibu pressure balanced diverter fixed shower...,Shower Panels,Home Improvement / Bathroom Remodel & Bathroom...,the malibu pressure balanced diverter fixed sh...,producttype : shower panel|spraypattern : rain...,3.0,4.5,2.0,"['producttype : shower panel', 'spraypattern :...",...,Bathroom Remodel & Bathroom Fixtures,Home Improvement / Bathroom Remodel & Bathro...,1.000000,shower head,shower head,shower head,malibu,bathroom,chrome sep stainless steel,Home Improvement / Bathroom Remodel & Bathroom...
42990,42990,emmeline 5 piece breakfast dining set,Dining Table Sets,Furniture / Kitchen & Dining Furniture / Dinin...,,basematerialdetails : steel| : gray wood|ofhar...,1314.0,4.5,864.0,"['basematerialdetails : steel', ' : gray wood'...",...,Kitchen & Dining Furniture,Furniture / Kitchen & Dining Furniture,0.691681,breakfast dining set,dining table,dining table,emmeline,dining room,,Furniture / Kitchen & Dining Furniture / Dinin...
42991,42991,maloney 3 piece pub table set,Dining Table Sets,Furniture / Kitchen & Dining Furniture / Dinin...,this pub table set includes 1 counter height t...,additionaltoolsrequirednotincluded : power dri...,49.0,4.0,41.0,['additionaltoolsrequirednotincluded : power d...,...,Kitchen & Dining Furniture,Fu

## Classify queries to item types

Below is a scaffold query classifier for you to do your work.

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Literal
import numpy as np

ItemType = Literal[
    "area rug", "accent pillow", "bed", "cocktail table", "floor & wall tile",
    "entertainment center", "kitchen mat", "sectional", "sofa", "patio sofa",
    "doormat", "furniture cushion", "wall clock", "garden statue",
    "kitchen island", "garment rack", "mattress pad",
    "loveseat", "armchair", "recliner", "coffee table", "end table",
    "tv stand", "media console", "bookshelf", "bed frame", "mattress", "nightstand",
    "dresser", "wardrobe", "chest of drawers", "dining table", "dining chair", "bar stool",
    "sideboard", "buffet", "bench", "office chair", "desk", "filing cabinet", "bookcase",
    "patio chair", "patio table", "outdoor sofa", "umbrella", "grill", "toolbox",
    "door knob", "door lock", "deadbolt", "light switch", "outlet", "extension cord",
    "smart bulb", "ceiling fan", "floor lamp", "table lamp", "chandelier", "rug",
    "curtains", "blinds", "shower curtain", "mirror", "wall art", "picture frame",
    "clock", "candle holder", "vase", "planter", "kitchen faucet", "sink", "toilet",
    "bathroom faucet", "chaise lounge",
    "shower head", "plunger", "broom", "dustpan", "mop", "bucket", "vacuum", "trash can",
    "recycling bin", "laundry basket", "ironing board", "drying rack", "cutlery", "slow cooker",
    "frying pan", "saucepan", "mixing bowl", "cutting board", "storage bin", "shelving unit",
    "no item type matches", "unknown", "ottoman", "comforter", "chair cushion", "refrigerator",
    "greenhouse", "crown molding", "vanity", "flag", "potted plant", "basket", "podium",
    "blanket", "anti-fatigue mat", "serving tray"
]

# Import MiniLM to encode via sentence transformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


@StoredLruCache(maxsize=50000)
def encode(text):
    return model.encode(text)


class Query(BaseModel):
    """
    Base model for search queries, containing common query attributes.
    """
    keywords: str = Field(
        ...,
        description="The original search query keywords sent in as input"
    )


class ItemTypeQuery(Query):
    """
    Represents the item type of relevant products in a search index for this query (ie dining table, bed, etc)
    """
    @property
    def similarity(self):
        """Compare item_type to item_type_unconstrained"""
        return np.dot(encode(self.item_type), encode(self.item_type_unconstrained))

    item_type: ItemType = Field(
        ...,
        description="The type of item of relevant products for the query from the provided list. Use 'no item type matches' if no item type matches the item"
    )
    item_type_unconstrained: str = Field(
        ...,
        description="The type of item of relevant products for the query, ie dining table, bed, etc"
    )

### Query prompt + enricher

We ask for item_type and item_type_unconstrained back from the prompt

In [ ]:
from cheat_at_search.enrich import AutoEnricher, ProductEnricher

item_type_enricher = AutoEnricher(
    model="openai/gpt-4.1-mini",
    system_prompt="You are a helpful furniture, hardware, and home-goods ecommerce shopping assistant that understands furniture search queries",
    response_model=ItemTypeQuery
)

def get_item_type_prompt(query) -> str:
    prompt = f"""
Given a search query, what item type descripbes the relevant product?

For 'item_type' Use 'no item type matches' if no listed item type matches the item.
For 'item_type_unconstrained' just extract any item type

Here's the search query to classify:

{query}
        """
    return prompt

# Don't provide an item type unless compelling evidence is in the search query that it matches to one of the listed item types.

def query_enricher(query: str) -> ItemTypeQuery:
    prompt = get_item_type_prompt(Query(keywords=query))
    return item_type_enricher.enrich(prompt)


query_enricher('sheffield pillow')

2025-09-01 13:29:19,264 - data_dir - INFO - Looking for openai in environment variables or globals...


INFO:data_dir:Looking for openai in environment variables or globals...


2025-09-01 13:29:19,276 - data_dir - INFO - Reading openai_api_key API key from /content/drive/MyDrive/cheat-at-search-data//keys.json


INFO:data_dir:Reading openai_api_key API key from /content/drive/MyDrive/cheat-at-search-data//keys.json


2025-09-01 13:29:19,607 - data_dir - INFO - openai_api_key key loaded successfully.


...

2025-09-01 13:29:22,271 - query_parser - ERROR - Error loading cache file /content/drive/MyDrive/cheat-at-search-data/enrich_cache/openaienricher_d38fd97127aa14cbccf7fd07232c1938_cache.pkl: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte


ERROR:query_parser:Error loading cache file /content/drive/MyDrive/cheat-at-search-data/enrich_cache/openaienricher_d38fd97127aa14cbccf7fd07232c1938_cache.pkl: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte


2025-09-01 13:29:22,273 - query_parser - ERROR - Starting with empty cache due to error.


ERROR:query_parser:Starting with empty cache due to error.


ItemTypeQuery(keywords='sheffield pillow', item_type='accent pillow', item_type_unconstrained='pillow')

## Measure the quality against ground truth

### Examine ground truth

The ground truth for the queries is the pre-enriched queries, based on aggregating known query-document relationships.

In [ ]:
ground_truth = enriched_queries[['item_type_same', 'item_type_unconstrained', 'query']]
ground_truth.loc[ground_truth['item_type_unconstrained'] == 'No Category Fits', 'item_type_unconstrained'] = 'unknown'
ground_truth

/tmp/ipython-input-3028597469.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ground_truth.loc[ground_truth['item_type_unconstrained'] == 'No Category Fits', 'item_type_unconstrained'] = 'unknown'


,item_type_same,item_type_unconstrained,query
0,unknown,unknown,12v dirt bike
1,unknown,unknown,3 1/2 inch drawer pull
2,area rug,unknown,3 piece rug set with runners
3,unknown,unknown,3/4 size mattress
4,vanity,unknown,30 inch bathroom vanity
...,...,...,...
475,tv stand,tv stand,wrought tv stand
476,bookshelf,ladder bookcase,zachary 72.5
477,bed frame,twin platform bed with trundle,zakariyah twin platform
478,unknown,unknown,zen


### Measure ground truth of classifier

Classify all queries than compare to ground truth

In [ ]:
import pandas as pd
from tqdm import tqdm
enriched_item_types = []
for query in tqdm(ground_truth['query']):
    item_type_predicted = query_enricher(query)
    item_type_as_dict = item_type_predicted.model_dump()
    similarity = item_type_predicted.similarity
    item_type_as_dict['item_type_sim'] = similarity
    item_type_as_dict['query'] = query
    enriched_item_types.append(item_type_as_dict)

enriched_item_types = pd.DataFrame(enriched_item_types)

100%|██████████| 480/480 [11:31<00:00,  1.44s/it]


### Measure against ground truth

Below we merge our predictions with the ground truth to build a datafrome holding both. We compute precision and coverage.

In [ ]:
q_enriched_item_types = enriched_item_types.add_suffix('_predicted').merge(ground_truth.add_suffix('_ground_truth'),
                                                                           left_on='keywords_predicted', right_on='query_ground_truth')
q_enriched_item_types = q_enriched_item_types.drop(columns=['query_ground_truth']).rename(columns={'keywords_predicted': 'query'})
q_enriched_item_types

,query,item_type_predicted,item_type_unconstrained_predicted,item_type_sim_predicted,query_predicted,item_type_same_ground_truth,item_type_unconstrained_ground_truth
0,12v dirt bike,no item type matches,dirt bike,0.052750,12v dirt bike,unknown,unknown
1,3 1/2 inch drawer pull,no item type matches,drawer pull,0.110507,3 1/2 inch drawer pull,unknown,unknown
2,3 piece rug set with runners,rug,rug set,0.815383,3 piece rug set with runners,area rug,unknown
3,3/4 size mattress,mattress,mattress,1.000000,3/4 size mattress,unknown,unknown
4,30 inch bathroom vanity,vanity,bathroom vanity,0.870768,30 inch bathroom vanity,vanity,unknown
...,...,...,...,...,...,...,...
475,wrought tv stand,tv stand,tv stand,1.000000,wrought tv stand,tv stand,tv stand
476,zachary 72.5,sofa,sofa,1.000000,zachary 72.5,bookshelf,ladder bookcase
477,zakariyah twin platform,bed frame,twin platform bed frame,0.724806,zakariyah twin platform,bed frame,twin platform bed with trundle
478,zen,no item type matches,zen decor,0.044414,zen,unknown,unknown


### Measure precision + coverage

Remember that precision is the percentage of products with predictions that are correct.

Precision is pretty poor. What can be done to improve it?

1. Only allow predictions where item_type_sim > a threshold, otherwise label as 'no item type matches'?
2. Change the above prediction to allow a list?

In [ ]:
THRESHOLD = -1.0
# THRESHOLD = 0.5 <--what happens if you change this to 0.5 or some other value?
THRESHOLD = 0.75
q_enriched_item_types.loc[q_enriched_item_types['item_type_sim_predicted'] < THRESHOLD, 'item_type_predicted'] = 'no item type matches'
q_enriched_item_types

,query,item_type_predicted,item_type_unconstrained_predicted,item_type_sim_predicted,query_predicted,item_type_same_ground_truth,item_type_unconstrained_ground_truth
0,12v dirt bike,no item type matches,dirt bike,0.052750,12v dirt bike,unknown,unknown
1,3 1/2 inch drawer pull,no item type matches,drawer pull,0.110507,3 1/2 inch drawer pull,unknown,unknown
2,3 piece rug set with runners,rug,rug set,0.815383,3 piece rug set with runners,area rug,unknown
3,3/4 size mattress,mattress,mattress,1.000000,3/4 size mattress,unknown,unknown
4,30 inch bathroom vanity,vanity,bathroom vanity,0.870768,30 inch bathroom vanity,vanity,unknown
...,...,...,...,...,...,...,...
475,wrought tv stand,tv stand,tv stand,1.000000,wrought tv stand,tv stand,tv stand
476,zachary 72.5,sofa,sofa,1.000000,zachary 72.5,bookshelf,ladder bookcase
477,zakariyah twin platform,no item type matches,twin platform bed frame,0.724806,zakariyah twin platform,bed frame,twin platform bed with trundle
478,zen,no item type matches,zen decor,0.044414,zen,unknown,unknown


In [ ]:
q_enriched_with_prediction = q_enriched_item_types[q_enriched_item_types['item_type_predicted'] != 'no item type matches']
precision = (q_enriched_with_prediction['item_type_predicted'] ==
             q_enriched_with_prediction['item_type_same_ground_truth']).mean()
coverage = len(q_enriched_with_prediction) / len(enriched_item_types)
print(f"Precision: {precision}")
print(f"Coverage: {coverage}")

Precision: 0.46111111111111114
Coverage: 0.375


In [ ]:
misclassifications = q_enriched_with_prediction[q_enriched_with_prediction['item_type_predicted']
                                                != q_enriched_with_prediction['item_type_same_ground_truth']]
misclassifications[['query', 'item_type_predicted', 'item_type_same_ground_truth']]

,query,item_type_predicted,item_type_same_ground_truth
2,3 piece rug set with runners,rug,area rug
3,3/4 size mattress,mattress,unknown
7,48 inch bathroom vanity with trough sink,vanity,unknown
10,5 gang light switch,light switch,unknown
11,54 in bench cushion,furniture cushion,unknown
...,...,...,...
453,wayfair comforters,comforter,unknown
454,wayfair sleep zippered,mattress pad,unknown
457,westling coffee table,coffee table,unknown
460,white bathroom vanity black hardware,vanity,unknown


In [ ]:
correct_classifications = q_enriched_with_prediction[q_enriched_with_prediction['item_type_predicted']
                                                     == q_enriched_with_prediction['item_type_same_ground_truth']]
correct_classifications[['query', 'item_type_predicted', 'item_type_same_ground_truth']]

,query,item_type_predicted,item_type_same_ground_truth
4,30 inch bathroom vanity,vanity,vanity
12,7 draw white dresser,dresser,dresser
14,7qt slow cooker,slow cooker,slow cooker
16,accent chair recliner,recliner,recliner
25,alyse 8 light,chandelier,chandelier
...,...,...,...
467,wood bar stools,bar stool,bar stool
468,wood coffee table set by storage,coffee table,coffee table
472,worn leather office chair,office chair,office chair
475,wrought tv stand,tv stand,tv stand


## Item Type search strategy

Just as previously, we add a constant boost to the underlying BM25 scores when the query's item type matches the product's

In [ ]:
from searcharray import SearchArray
from cheat_at_search.tokenizers import snowball_tokenizer
from cheat_at_search.strategy.strategy import SearchStrategy
import numpy as np


class ItemTypeSearch(SearchStrategy):
    def __init__(self,
                 products,
                 query_to_item_type,
                 name_boost=9.3,
                 description_boost=4.1,
                 item_type_boost=100):
        super().__init__(products)
        self.index = products
        self.index['product_name_snowball'] = SearchArray.index(
            products['product_name'], snowball_tokenizer)
        self.index['product_description_snowball'] = SearchArray.index(
            products['product_description'], snowball_tokenizer)

        cat_split = products['category hierarchy'].fillna('').str.split("/")

        self.index['item_type_snowball'] = SearchArray.index(
            products['item_type_same'], snowball_tokenizer
        )

        self.query_to_item_type = query_to_item_type
        self.name_boost = name_boost
        self.description_boost = description_boost
        self.item_type_boost = item_type_boost

    def search(self, query, k=10):
        """Dumb baseline lexical search, but add a constant boost when
           the desired category or subcategory"""
        bm25_scores = np.zeros(len(self.index))
        item_type = self.query_to_item_type(query).item_type
        tokenized = snowball_tokenizer(query)

        # ****
        # Baseline BM25 search from before
        for token in tokenized:
            bm25_scores += self.index['product_name_snowball'].array.score(token) * self.name_boost
            bm25_scores += self.index['product_description_snowball'].array.score(
                token) * self.description_boost

        # ****
        # If there's a item_type mentioned, boost that by a constant amount
        if item_type and item_type != 'no item type matches':
            tokenized_item_type = snowball_tokenizer(item_type)
            item_type_match = np.ones(len(self.index))
            if tokenized_item_type:
                item_type_match = self.index['item_type_snowball'].array.score(tokenized_item_type) > 0
            bm25_scores[item_type_match] += self.item_type_boost

        top_k = np.argsort(-bm25_scores)[:k]
        scores = bm25_scores[top_k]

        return top_k, scores

item_type_search = ItemTypeSearch(enriched_products, query_to_item_type=query_enricher)
graded_item_type = run_strategy(item_type_search)

2025-09-01 13:47:18,941 - searcharray.indexing - INFO - Indexing begins w/ 4 workers


INFO:searcharray.indexing:Indexing begins w/ 4 workers


2025-09-01 13:47:18,954 - searcharray.indexing - INFO - 0 Batch Start tokenization


INFO:searcharray.indexing:0 Batch Start tokenization


2025-09-01 13:47:18,958 - searcharray.indexing - INFO - Tokenizing 42994 documents


...

INFO:searcharray.indexing:Batch tokenization complete


2025-09-01 13:47:30,064 - searcharray.indexing - INFO - (main thread) Processing 1 batch results


INFO:searcharray.indexing:(main thread) Processing 1 batch results


2025-09-01 13:47:30,106 - searcharray.indexing - INFO - Indexing from tokenization complete


INFO:searcharray.indexing:Indexing from tokenization complete
Searching: 100%|██████████| 480/480 [00:16<00:00, 29.85it/s]


### Analyze the results

We observe a reasonable improvement by adding item type. If we can predict item type reasonably well on queries, we have a chance to meaningfully improve search.

We further observe that MANY queries were improved, so the impact is broad.

In [ ]:
ndcgs(graded_item_type).mean(), ndcgs(graded_bm25).mean()

(np.float64(0.5343170904273735), np.float64(0.5411098691836396))

In [ ]:
deltas = ndcg_delta(graded_item_type, graded_bm25)
print(len(deltas))
deltas[deltas != 0]

204


,ndcg
query,
acrylic clear chair,0.711286
drum picture,0.699905
hardwood beds,0.651944
outdoor lounge chair,0.623086
pull out sleeper loveseat,0.591790
...,...
dining table vinyl cloth,-0.745427
above toilet cabinet,-0.836472
breakfast bar table,-0.850065


In [ ]:
deltas[deltas < -0.1]

,ndcg
query,
broadway lever,-0.109019
cake cutlery,-0.110554
wall shelves,-0.110644
wire basket with dividers,-0.113806
black 5 drawer dresser by guilford,-0.113806
upholstered bed,-0.113806
bistro table and chairs,-0.114077
wood floor protector mat,-0.118999
enclosed shoe rack,-0.124690


In [ ]:
QUERY = 'sheffield home bath set'
graded_item_type[graded_item_type['query'] == QUERY][['product_name', 'product_description', 'grade', 'item_type', 'score']]

,product_name,product_description,grade,item_type,score
750,sheffield 72 '' double bathroom vanity set,distinctive styling and elegant lines come tog...,1.0,dresser,69.806143
751,sheffield 48 '' single bathroom vanity set,distinctive styling and elegant lines come tog...,2.0,vanity,67.618840
752,sheffield 60 '' double bathroom vanity set,distinctive styling and elegant lines come tog...,1.0,vanity,67.618840
753,sheffield 30 '' single bathroom vanity set,distinctive styling and elegant lines come tog...,1.0,vanity,67.618840
754,sheffield 36 '' single bathroom vanity set,distinctive styling and elegant lines come tog...,1.0,vanity,67.618840
755,sheffield 80 '' double bathroom vanity set,distinctive styling and elegant lines come tog...,1.0,dresser,67.618840
756,sheffield 60 '' single bathroom vanity set,distinctive styling and elegant lines come tog...,1.0,vanity,67.618840
757,sheffield shower curtain hooks,"featuring an attractive design , the carnation...",0.0,shower curtain,66.074451
758,sheffield resin shower curtain hooks,bring a subtle elegance to your bathroom with ...,0.0,shower curtain,63.268106
759,mcginty 3 piece bath set,stylish and contemporary geometric design by c...,1.0,bathroom faucet,54.024371


In [ ]:
graded_bm25[graded_bm25['query'] == QUERY][['product_id', 'product_name', 'product_description', 'grade', 'score']]

,product_id,product_name,product_description,grade,score
750,39055,sheffield 72 '' double bathroom vanity set,distinctive styling and elegant lines come tog...,1.0,69.806143
751,39056,sheffield 48 '' single bathroom vanity set,distinctive styling and elegant lines come tog...,2.0,67.618840
752,39052,sheffield 60 '' double bathroom vanity set,distinctive styling and elegant lines come tog...,1.0,67.618840
753,39054,sheffield 30 '' single bathroom vanity set,distinctive styling and elegant lines come tog...,1.0,67.618840
754,39022,sheffield 36 '' single bathroom vanity set,distinctive styling and elegant lines come tog...,1.0,67.618840
755,39053,sheffield 80 '' double bathroom vanity set,distinctive styling and elegant lines come tog...,1.0,67.618840
756,39021,sheffield 60 '' single bathroom vanity set,distinctive styling and elegant lines come tog...,1.0,67.618840
757,31131,sheffield shower curtain hooks,"featuring an attractive design , the carnation...",0.0,66.074451
758,3194,sheffield resin shower curtain hooks,bring a subtle elegance to your bathroom with ...,0.0,63.268106
759,6039,mcginty 3 piece bath set,stylish and contemporary geometric design by c...,1.0,54.024371


In [ ]:
products[products['product_id'] == 39056].iloc[0]

,39056
product_id,39056
product_name,sheffield 48 '' single bathroom vanity set
product_class,Vanities
category hierarchy,Home Improvement / Bathroom Remodel & Bathroom...
product_description,distinctive styling and elegant lines come tog...
product_features,whatisap-trap : a p-trap holds water to preven...
rating_count,2.0
average_rating,5.0
review_count,2.0
features,[whatisap-trap : a p-trap holds water to preve...


In [ ]:
enriched_queries

,query,room,item_type_same,branded_terms,category hierarchy,product_class,category,sub_category,materials,item_type_unconstrained,query_classification,corrected_query,classification
0,12v dirt bike,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,No Category Fits,Outdoor / Outdoor Recreation / Backyard Play /...,12v dirt bike,Outdoor / Outdoor Recreation / Backyard Play /...
1,3 1/2 inch drawer pull,unknown,unknown,unknown,unknown,Cabinet and Drawer Pulls,Home Improvement,Hardware,unknown,No Category Fits,Home Improvement / Hardware / Cabinet Hardware...,3 1/2 inch drawer pull,Home Improvement / Hardware / Cabinet Hardware...
2,3 piece rug set with runners,unknown,area rug,unknown,unknown,Area Rugs,Rugs,unknown,unknown,No Category Fits,Rugs / Area Rugs sep Rugs / Kitchen Mats sep R...,3 piece rug set with runners,Rugs / Area Rugs sep Rugs / Kitchen Mats sep R...
3,3/4 size mattress,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,No Category Fits,Furniture / Bedroom Furniture / Beds & Headboa...,3/4 size mattress,Furniture / Bedroom Furniture / Beds & Headboa...
4,30 inch bathroom vanity,bathroom,vanity,unknown,unknown,Vanities,Home Improvement,Bathroom Remodel & Bathroom Fixtures,unknown,No Category Fits,Home Improvement / Bathroom Remodel & Bathroom...,30 inch bathroom vanity,Home Improvement / Bathroom Remodel & Bathroom...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,wrought tv stand,living room,tv stand,unknown,Furniture / Living Room Furniture / TV Stands ...,TV Stands & Entertainment Centers,Furniture,Living Room Furniture,unknown,tv stand,Furniture / Living Room Furniture / TV Stands ...,wrought tv stand,Furniture / Living Room Furniture / TV Stands ...
476,zachary 72.5,living room,bookshelf,zachary,Furniture / Living Room Furniture / Bookcases,Teen Bookcases|Bookcases,Furniture,Living Room Furniture,steel,ladder bookcase,Storage & Organization / Wall Shelving & Organ...,zachary 72.5,Storage & Organization / Wall Shelving & Organ...
477,zakariyah twin platform,bedroom,bed frame,twin platform bed with trundle sep zakariyah,Baby & Kids / Toddler & Kids Bedroom Furniture...,Beds|Kids Beds,Baby & Kids,Toddler & Kids Bedroom Furniture,unknown,twin platform bed with trundle,Furniture / Bedroom Furniture / Beds & Headboa...,zakariyah twin platform,Furniture / Bedroom Furniture / Beds & Headboa...
478,zen,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,No Category Fits,Décor & Pillows / Home Accessories / Decorativ...,zen,Décor & Pillows / Home Accessories / Decorativ...
